In [22]:
import pandas as pd
import numpy as np

DATA_PATH = "../DataSets/processed/renewal_risk_score_dataset.csv"
df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


(60000, 28)


,customer_id,policy_id,policy_start_date,policy_expiry_date,renewal_date,total_claims,total_claim_amount,avg_claim_amount,max_claim_amount,major_claim_count,...,had_major_claim_last_1_year,years_since_last_claim,frequent_claimer_flag,high_long_term_frequency_flag,high_severity_claimer_flag,small_frequent_claims_flag,freq_score,sev_score,recency_score,risk_score
0,C000001,P000001,2023-09-09,2024-09-09,2024-09-09,2.0,513817.0,256908.5,441899.0,0.0,...,0.0,0.114990,0,0,0,0,0.5,0.866250,0.977264,7.72
1,C000002,P000002,2023-02-14,2024-02-14,2024-02-14,2.0,1826187.0,913093.5,1322535.0,1.0,...,1.0,0.342231,0,0,0,0,0.5,0.939303,0.933844,22.83
2,C000003,P000003,2023-09-12,2024-09-12,2024-09-12,2.0,4155749.0,2077874.5,2118271.0,2.0,...,1.0,0.772074,0,0,0,0,0.5,0.970694,0.856917,27.43
3,C000004,P000004,2024-10-09,2025-10-09,2025-10-09,2.0,196571.0,98285.5,118691.0,0.0,...,0.0,0.380561,0,0,0,0,0.5,0.778647,0.926712,5.82
4,C000005,P000005,2024-02-29,2025-02-28,2025-02-28,0.0,0.0,0.0,0.0,0.0,...,0.0,999.000000,0,0,0,0,0.0,0.000000,0.135335,0.00


In [23]:
FEATURES = [
    "freq_score",
    "sev_score",
    "recency_score",
    "claims_last_1_year",
    "claims_last_3_years",
    "had_major_claim_last_1_year",
    "small_frequent_claims_flag",
    "years_since_last_claim"
]

TARGET = "total_claim_amount_last_1_year"

X = df[FEATURES]
y = df[TARGET]


In [13]:
import numpy as np
import pandas as pd

# If X_train is a DataFrame
X_train_np = X_train.to_numpy(dtype=np.float64)
X_test_np  = X_test.to_numpy(dtype=np.float64)

# If y_train is a Series/DataFrame -> make it 1D
y_train_np = np.asarray(y_train).reshape(-1).astype(np.float64)
y_test_np  = np.asarray(y_test).reshape(-1).astype(np.float64)

print("X_train:", X_train_np.shape, X_train_np.dtype)
print("y_train:", y_train_np.shape, y_train_np.dtype)


X_train: (48000, 8) float64
y_train: (48000,) float64


In [15]:
print("X_train NaN:", np.isnan(X_train_np).sum())
print("y_train NaN:", np.isnan(y_train_np).sum())

print("X_train inf:", np.isinf(X_train_np).sum())
print("y_train inf:", np.isinf(y_train_np).sum())


X_train NaN: 0
y_train NaN: 0
X_train inf: 0
y_train inf: 0


In [25]:
from sklearn.tree import DecisionTreeRegressor
from ngboost import NGBRegressor
from ngboost.distns import Normal
from ngboost.scores import LogScore

base_learner = DecisionTreeRegressor(
    max_depth=3,
    min_samples_leaf=30,
    random_state=42
)

ngb = NGBRegressor(
    Base=base_learner,
    Dist=Normal,
    Score=LogScore,
    n_estimators=600,
    learning_rate=0.03,
    random_state=42,
    verbose=True,
    natural_gradient=False
)
ngb.fit(X_train_np, y_train_np)



[iter 0] loss=15.8431 val_loss=0.0000 scale=0.5000 norm=0.4603
[iter 100] loss=15.5080 val_loss=0.0000 scale=1.0000 norm=0.3617
[iter 200] loss=15.4815 val_loss=0.0000 scale=1.0000 norm=0.3264
[iter 300] loss=15.4745 val_loss=0.0000 scale=0.5000 norm=0.1585
[iter 400] loss=15.4723 val_loss=0.0000 scale=1.0000 norm=0.3108
[iter 500] loss=15.4711 val_loss=0.0000 scale=0.5000 norm=0.1540


,Dist,<class 'ngboo...ormal.Normal'>
,Score,<class 'ngboo...res.LogScore'>
,Base,DecisionTreeR...ndom_state=42)
,natural_gradient,False
,n_estimators,600
,learning_rate,0.03
,minibatch_frac,1.0
,col_sample,1.0
,verbose,True
,verbose_eval,100
,tol,0.0001


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pred_dist = ngb.pred_dist(X_test_np)
pred_mean = pred_dist.mean()

mae  = mean_absolute_error(y_test_np, pred_mean)
rmse = np.sqrt(mean_squared_error(y_test_np, pred_mean))
r2   = r2_score(y_test_np, pred_mean)

print("MAE :", mae)
print("RMSE:", rmse)
print("R²  :", r2)


MAE : 1480097.2022799065
RMSE: 1822543.397386149
R²  : -4.609843602110253e-05


In [27]:
print(type(y_train), getattr(y_train, "shape", None))
print(np.asarray(y_train).shape)


<class 'pandas.core.series.Series'> (48000,)
(48000,)


In [30]:
import joblib
from pathlib import Path

MODEL_DIR = Path("../models/")
MODEL_DIR.mkdir(exist_ok=True)

MODEL_PATH = MODEL_DIR / "ngboost_renewal_bundle.pkl"

joblib.dump(
    {
        "model": ngb,
        "feature_columns": X_train.columns.tolist(), 
        "target": "total_claim_amount_last_1_year",
        "model_type": "NGBoost_Normal",
        "purpose": "Renewal risk – expected claim amount"
    },
    MODEL_PATH
)

print("Model saved to:", MODEL_PATH)


Model saved to: ..\models\ngboost_renewal_bundle.pkl


In [34]:
import joblib

bundle = joblib.load("../models/ngboost_renewal_bundle.pkl")
model = bundle["model"]

print("Model type:", type(model))
print("Dist:", getattr(model, "Dist", None))
print("Has pred_dist:", hasattr(model, "pred_dist"))


Model type: <class 'ngboost.api.NGBRegressor'>
Dist: <class 'ngboost.distns.normal.Normal'>
Has pred_dist: True
